In [1]:
"""
Ensemble predicitons from different models

Linear assignment ensembling 
nearest points between predictions.
Hungalian algorithm.

Credits to: https://www.kaggle.com/c/pku-autonomous-driving/discussion/127034
"""

from scipy.optimize import linear_sum_assignment
import pandas as pd
from tqdm import tqdm
import numpy as np
from math import sin, cos
from sklearn.model_selection import train_test_split

In [2]:
# convert euler angle to rotation matrix
def euler_to_Rot(yaw, pitch, roll):
    Y = np.array([[cos(yaw), 0, sin(yaw)],
                  [0, 1, 0],
                  [-sin(yaw), 0, cos(yaw)]])
    P = np.array([[1, 0, 0],
                  [0, cos(pitch), -sin(pitch)],
                  [0, sin(pitch), cos(pitch)]])
    R = np.array([[cos(roll), -sin(roll), 0],
                  [sin(roll), cos(roll), 0],
                  [0, 0, 1]])
    return np.dot(Y, np.dot(P, R))

In [3]:
## Function to extract position values
def str2coords(s, names=['id', 'yaw', 'pitch', 'roll', 'x', 'y', 'z']):
    '''
    Input:
        s: PredictionString (e.g. from train dataframe)
        names: array of what to extract from the string
    Output:
        list of dicts with keys from `names`
    '''
    coords = []
    for l in np.array(s.split()).reshape([-1, 7]):
        coords.append(dict(zip(names, l.astype('float'))))
        if 'id' in coords[-1]:
            coords[-1]['id'] = int(coords[-1]['id'])
    return coords

In [4]:
def get_imgcoords(s):
    '''
    Input is a PredictionString (e.g. from train dataframe)
    Output is two arrays:
        xs: x coordinates in the image (row)
        ys: y coordinates in the image (column)
    '''
    coords = str2coords(s)
    xs = [c['x'] for c in coords]
    ys = [c['y'] for c in coords]
    zs = [c['z'] for c in coords]
    P = np.array(list(zip(xs, ys, zs))).T
    img_p = np.dot(camera_matrix, P).T
    img_p[:, 0] /= img_p[:, 2]
    img_p[:, 1] /= img_p[:, 2]
    img_xs = img_p[:, 0]
    img_ys = img_p[:, 1]
    img_zs = img_p[:, 2] # z = Distance from the camera
    return img_xs, img_ys, img_zs

In [5]:
sub1 = pd.read_csv('../Documents/predictions-f.csv')
y1 = sub1['PredictionString'].str.split(' ').values
X1 = sub1['ImageId'].values

sub2 = pd.read_csv('../Documents/predictions-f2.csv')
y2 = sub2['PredictionString'].str.split(' ').values
X2 = sub2['ImageId'].values

for idx in tqdm(range(len(sub1)), position=0):
    if str(np.nan) != str(y1[idx]) and str(np.nan) != str(y2[idx]):
        label1 = np.array(y1[idx]).reshape(-1, 7).astype(float)
        label2 = np.array(y2[idx]).reshape(-1, 7).astype(float)
        center_points1 = get_imgcoords(label1) # [N, 3], (img_x, img_y, img_z)
        center_points2 = get_imgcoords(label2)

        cost_matrix = np.zeros([len(center_points1), len(center_points1)])
        for idx1, i in enumerate(center_points1):
            for idx2, j in enumerate(center_points2):
                cost_matrix[idx1, idx2] = np.linalg.norm(i - j)

        match1, match2 = linear_sum_assignment(cost_matrix)
        for i, j in zip(match1, match2):
            if cost_matrix[i, j] < distance_th:
                if np.abs(label1[i][1] - label2[j][1]) < yaw_th:
                    label1[j] = (label1[i] + label2[j]) / 2
        label1[idx] = np.concatenate(tmp1).astype(str)


  0%|          | 0/2021 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [ ]:
test = pd.read_csv(PATH + 'sample_submission.csv')
test['PredictionString'] = label1
# test.to_csv('predictions.csv', index=False)
test.to_csv('ensemble1.csv', index=False)
test.head()